In [4]:
import transformers
from datasets import load_dataset, load_metric
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

datasets = load_dataset("conll2003")
label_list = datasets["train"].features["ner_tags"].feature.names
label_list

Using the latest cached version of the module from /home/v-labsai-kaustubh-roy/.cache/huggingface/modules/datasets_modules/datasets/conll2003/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98 (last modified on Wed Aug 24 16:24:25 2022) since it couldn't be found locally at conll2003., or remotely on the Hugging Face Hub.
Reusing dataset conll2003 (/home/v-labsai-kaustubh-roy/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98)
100%|██████████| 3/3 [00:00<00:00, 326.88it/s]


['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [1]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
label_all_tokens = True

/home/v-labsai-kaustubh-roy/anaconda3/envs/pythonProject3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [5]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Loading cached processed dataset at /home/v-labsai-kaustubh-roy/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98/cache-51944836faf57e7b.arrow
100%|██████████| 4/4 [00:00<00:00, 11.28ba/s]
Loading cached processed dataset at /home/v-labsai-kaustubh-roy/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98/cache-85e7909df6fa3b46.arrow


In [6]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
base_model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN t

In [7]:
from transformers import DistilBertConfig, BertConfig, DistilBertModel, BertModel, BertForTokenClassification, DistilBertForTokenClassification
from torch import nn
import numpy as np
config = DistilBertConfig()
my_config = DistilBertConfig(n_layers=3)
my_config.num_labels= len(label_list)
model = DistilBertForTokenClassification.from_pretrained("distilbert-base-uncased", config=my_config)
num_labels = len(label_list)
model.distilbert.transformer.layer[0]=base_model.distilbert.transformer.layer[0]
model.distilbert.transformer.layer[1]=base_model.distilbert.transformer.layer[2]
model.distilbert.transformer.layer[2]=base_model.distilbert.transformer.layer[4]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['distilbert.transformer.layer.4.sa_layer_norm.bias', 'vocab_transform.weight', 'distilbert.transformer.layer.4.attention.q_lin.weight', 'distilbert.transformer.layer.5.ffn.lin2.weight', 'distilbert.transformer.layer.4.output_layer_norm.weight', 'distilbert.transformer.layer.3.output_layer_norm.bias', 'distilbert.transformer.layer.5.sa_layer_norm.bias', 'distilbert.transformer.layer.5.attention.out_lin.weight', 'distilbert.transformer.layer.4.sa_layer_norm.weight', 'distilbert.transformer.layer.5.attention.q_lin.weight', 'distilbert.transformer.layer.4.ffn.lin2.weight', 'distilbert.transformer.layer.3.sa_layer_norm.bias', 'distilbert.transformer.layer.5.attention.v_lin.bias', 'distilbert.transformer.layer.3.attention.q_lin.weight', 'vocab_layer_norm.weight', 'distilbert.transformer.layer.3.ffn.lin1.weight', 'distilbert.transformer.layer.5.attention.q_lin.bia

In [9]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer)

In [10]:
args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)
args = TrainingArguments()

In [11]:
example = datasets["train"][4]
print(example["tokens"])

['Germany', "'s", 'representative', 'to', 'the', 'European', 'Union', "'s", 'veterinary', 'committee', 'Werner', 'Zwingmann', 'said', 'on', 'Wednesday', 'consumers', 'should', 'buy', 'sheepmeat', 'from', 'countries', 'other', 'than', 'Britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.']


In [12]:
metric = load_metric("seqeval")
labels = [label_list[i] for i in example["ner_tags"]]
metric.compute(predictions=[labels], references=[labels])

Using the latest cached version of the module from /home/v-labsai-kaustubh-roy/.cache/huggingface/modules/datasets_modules/metrics/seqeval/c8563af43bdce095d0f9e8b8b79c9c96d5ea5499b3bf66f90301c9cb82910f11 (last modified on Fri Aug 19 16:43:10 2022) since it couldn't be found locally at seqeval, or remotely on the Hugging Face Hub.


{'LOC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'PER': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [14]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [19]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [20]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: tokens, ner_tags, id, pos_tags, chunk_tags. If tokens, ner_tags, id, pos_tags, chunk_tags are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
/home/v-labsai-kaustubh-roy/anaconda3/envs/pythonProject3/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 14041
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2634
 19%|█▉        | 500/2634 [00:52<04:09,  8.56it/s]Savi

{'loss': 0.0545, 'learning_rate': 1.6203492786636296e-05, 'epoch': 0.57}


Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
 33%|███▎      | 877/2634 [01:34<03:17,  8.91it/s]The following columns in the evaluation set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: tokens, ner_tags, id, pos_tags, chunk_tags. If tokens, ner_tags, id, pos_tags, chunk_tags are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3250
  Batch size = 16

 33%|███▎      | 880/2634 [01:42<37:24,  1.28s/it]

{'eval_loss': 0.09163288027048111, 'eval_precision': 0.8761978191430775, 'eval_recall': 0.8899205727710034, 'eval_f1': 0.883005883005883, 'eval_accuracy': 0.9743434953214609, 'eval_runtime': 7.1432, 'eval_samples_per_second': 454.979, 'eval_steps_per_second': 28.559, 'epoch': 1.0}


 38%|███▊      | 1000/2634 [01:54<02:50,  9.56it/s]Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json


...

Model weights saved in ./results/checkpoint-2000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-2000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-2000/special_tokens_map.json
 95%|█████████▍| 2500/2634 [04:47<00:14,  9.03it/s]Saving model checkpoint to ./results/checkpoint-2500
Configuration saved in ./results/checkpoint-2500/config.json


{'loss': 0.0214, 'learning_rate': 1.0174639331814731e-06, 'epoch': 2.85}


Model weights saved in ./results/checkpoint-2500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-2500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-2500/special_tokens_map.json
100%|██████████| 2634/2634 [05:04<00:00, 10.36it/s]The following columns in the evaluation set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: tokens, ner_tags, id, pos_tags, chunk_tags. If tokens, ner_tags, id, pos_tags, chunk_tags are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3250
  Batch size = 16

100%|██████████| 2634/2634 [05:12<00:00, 10.36it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 2634/2634 [05:12<00:00,  8.44it/s]

{'eval_loss': 0.09263940155506134, 'eval_precision': 0.8834983498349835, 'eval_recall': 0.8984226423537308, 'eval_f1': 0.8908979976704199, 'eval_accuracy': 0.9758685878596279, 'eval_runtime': 7.142, 'eval_samples_per_second': 455.056, 'eval_steps_per_second': 28.564, 'epoch': 3.0}
{'train_runtime': 312.1083, 'train_samples_per_second': 134.963, 'train_steps_per_second': 8.439, 'train_loss': 0.033569295476943865, 'epoch': 3.0}


TrainOutput(global_step=2634, training_loss=0.033569295476943865, metrics={'train_runtime': 312.1083, 'train_samples_per_second': 134.963, 'train_steps_per_second': 8.439, 'train_loss': 0.033569295476943865, 'epoch': 3.0})

In [21]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: tokens, ner_tags, id, pos_tags, chunk_tags. If tokens, ner_tags, id, pos_tags, chunk_tags are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3250
  Batch size = 16
100%|██████████| 204/204 [00:07<00:00, 28.54it/s]


{'eval_loss': 0.09263940155506134,
 'eval_precision': 0.8834983498349835,
 'eval_recall': 0.8984226423537308,
 'eval_f1': 0.8908979976704199,
 'eval_accuracy': 0.9758685878596279,
 'eval_runtime': 7.1851,
 'eval_samples_per_second': 452.328,
 'eval_steps_per_second': 28.392,
 'epoch': 3.0}

In [18]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: chunk_tags, id, tokens, pos_tags, ner_tags. If chunk_tags, id, tokens, pos_tags, ner_tags are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3250
  Batch size = 16
100%|█████████▉| 203/204 [00:11<00:00, 14.75it/s]

{'LOC': {'precision': 0.949305294780323,
  'recall': 0.9656226126814362,
  'f1': 0.9573944328725621,
  'number': 2618},
 'MISC': {'precision': 0.7958860759493671,
  'recall': 0.8172217709179529,
  'f1': 0.8064128256513027,
  'number': 1231},
 'ORG': {'precision': 0.9020368574199806,
  'recall': 0.9046692607003891,
  'f1': 0.9033511413307432,
  'number': 2056},
 'PER': {'precision': 0.9766524169681026,
  'recall': 0.978905735003296,
  'f1': 0.9777777777777779,
  'number': 3034},
 'overall_precision': 0.9262458471760797,
 'overall_recall': 0.9356751314464705,
 'overall_f1': 0.9309366130558184,
 'overall_accuracy': 0.9834622777892513}

: 